In [1]:
import os
import utm
import gdal
import time
import warnings
from osgeo import gdal
from osmapi import OsmApi
#http://python-overpy.readthedocs.io/en/latest/introduction.html
import overpy
# http://overpass-turbo.eu/# <- construct nice queries
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
# GDAL does not use python exceptions by default
gdal.UseExceptions()
api = overpy.Overpass()

In [2]:
#run this to get something we can render or use the visual product
#gdal_translate -ot Byte -scale 0 4096 0 255 /home/kat/code/hacking/osm/20170222_231253_0c27/20170222_231253_0c27_3B_AnalyticMS.tif test.tif

In [3]:
infile = "/home/katscott/code/hacking/PythonFromSpace/20170211_181115_0e16/20170211_181115_0e16_3B_Visual.tif"
#infile = "/home/kat/code/hacking/PythonFromSpace/test.tif"
gtif = gdal.Open(infile)
print gtif.GetProjectionRef()
arr = gtif.ReadAsArray()
trans = gtif.GetGeoTransform()
print trans
extent = (trans[0], trans[0] + gtif.RasterXSize*trans[1],
          trans[3] + gtif.RasterYSize*trans[5], trans[3])
pos = [5000,1900] # this is the pixel we want info about


PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]
(540504.0, 3.0, 0.0, 4191378.0, 0.0, -3.0)


In [4]:
def pixel2utm(ds, x, y):
    """
    Returns utm coordinates from pixel x, y coords
    """
    xoff, a, b, yoff, d, e = ds.GetGeoTransform()
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [5]:
ds = gdal.Open(infile)
# take the GDAL info and make it into UTM
my_utm = pixel2utm(ds,pos[0],pos[1])
# convert UTM into Lat Long
# need to figure out how to get zone info
my_lla = utm.to_latlon(my_utm[0],my_utm[1],10,"N")
# do the lat long look up from OSM
geolocator = Nominatim()
# reverse look up the are based on lat lon
location = geolocator.reverse("{0},{1}".format(my_lla[0],my_lla[1]))
# print location info
print location.address
print location.raw
# get the OSM ID info
osm_id = int(location.raw["place_id"])
print osm_id
# create an interface to the OSM API
MyApi = OsmApi()
# Look up our position 
print MyApi.NodeGet(osm_id)


SF Winery, 200, California Avenue, SF, California, 94130, United States of America
{u'display_name': u'SF Winery, 200, California Avenue, SF, California, 94130, United States of America', u'place_id': u'69202063', u'lon': u'-122.36980175', u'boundingbox': [u'37.8168224', u'37.8179618', u'-122.3703775', u'-122.369226'], u'osm_type': u'way', u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright', u'osm_id': u'24274429', u'lat': u'37.8173921', u'address': {u'building': u'SF Winery', u'city': u'SF', u'house_number': u'200', u'country': u'United States of America', u'county': u'SF', u'state': u'California', u'postcode': u'94130', u'country_code': u'us', u'road': u'California Avenue'}}
69202063


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:334: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


{u'changeset': 3353549, u'uid': 147510, u'timestamp': datetime.datetime(2009, 12, 12, 3, 38, 31), u'lon': -84.562955, u'visible': True, u'version': 2, u'user': u'woodpeck_fixbot', u'lat': 33.592651, u'tag': {}, u'id': 69202063}


In [6]:
plt.imshow(arr[:3,:,:].transpose((1, 2, 0)))#, extent=extent)
fig = plt.gcf()
plt.annotate(location.address, xy=pos, xycoords='data',
             xytext=(0.25, 0.5), textcoords='figure fraction',color="red",
             arrowprops=dict(arrowstyle="->"))
fig.set_size_inches(18.5, 10.5)
plt.show()

In [7]:
# we have the utm of our scene
# which we move to lat long
# now if we can make a bounding box
# we can query all of the coast in that box. 
# Buil the bounding box for the scene
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
xmin = gt[0]
ymin = gt[3] + width * gt[4] + height * gt[5]
xmax = gt[0] + width * gt[1] + height * gt[2]
ymax = gt[3]
array = [xmin, ymin, xmax, ymax]
# This is in utm convert to LLA
tl = utm.to_latlon(xmin,ymin,10,"N")
br = utm.to_latlon(xmax,ymax,10,"N")
# here is the base query
# this thing wants things in the very specific
# format s,w,n,e lat,lon,lat,lonmultpoly
query = """
    way({0},{1},{2},{3})["natural"="coastline"];
    (._;>;);
    out body;
    """
place = query.format(tl[0],tl[1],br[0],br[1])   
result = api.query(place)
print result.nodes

[<overpy.Node id=31788861 lat=37.8129190 lon=-122.3605931>, <overpy.Node id=31788875 lat=37.8740866 lon=-122.4607248>, <overpy.Node id=31789063 lat=37.8337768 lon=-122.2952954>, <overpy.Node id=31789123 lat=37.8249917 lon=-122.4920456>, <overpy.Node id=31789189 lat=37.8368874 lon=-122.3069237>, <overpy.Node id=31789275 lat=37.7911048 lon=-122.2911340>, <overpy.Node id=31789302 lat=37.7877821 lon=-122.2600318>, <overpy.Node id=31789471 lat=37.7836415 lon=-122.2444084>, <overpy.Node id=31789507 lat=37.7771140 lon=-122.3902912>, <overpy.Node id=31789868 lat=37.8636458 lon=-122.4884330>, <overpy.Node id=31790002 lat=37.8596191 lon=-122.4840513>, <overpy.Node id=31790160 lat=37.8682491 lon=-122.4618443>, <overpy.Node id=31790223 lat=37.8083172 lon=-122.4281535>, <overpy.Node id=31790252 lat=37.7971545 lon=-122.3305408>, <overpy.Node id=31790370 lat=37.8515323 lon=-122.4787522>, <overpy.Node id=31790562 lat=37.8052641 lon=-122.3263984>, <overpy.Node id=31790845 lat=37.8672140 lon=-122.434392

In [8]:
utm_points = []
for node in result.nodes:
    utm_points.append(utm.from_latlon(node.lat,node.lon))
utm_points
node.from_xml?

In [9]:
    print node.from_xml

<bound method type.from_xml of <class 'overpy.Node'>>


In [10]:
import cv2 
band = gtif.GetRasterBand(1)

cols = gtif.RasterXSize
rows = gtif.RasterYSize

transform = gtif.GetGeoTransform()

xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

data = band.ReadAsArray(0, 0, cols, rows)
arr = gtif.ReadAsArray()
temp = arr[0:3,:,:]
cv2.line(temp, (0,0), (300,300), (0,255,0))

for point in utm_points:
    col = int((point[0] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[1] ) / pixelHeight)
    if( col > 0 and row < 4547 and row > 0 and col < 8929 ):    
        arr[0,row,col] = 255
        arr[1,row,col] = 0
        arr[2,row,col] = 0

plt.imshow(temp[:3,:,:].transpose((1, 2, 0)))#, extent=extent)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()